In [12]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("LAB03_DataFrames_SQL")
    .master("local[*]")
    .getOrCreate()
)

spark

In [13]:
spark.conf.set("spark.sql.shuffle.partitions", "8")  # shuffle partitions (handy in local mode)
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")  # optional speedup via Arrow

In [14]:
file_path = "/Users/abidhiafahmed/projets/realtime-spark/data/ecommerce_kaggle/OnlineRetail.csv"  # adapt USERNAME and filename

df_raw = (
    spark.read
    .option("header", "true")       # first line contains headers
    .option("inferSchema", "true")  # infer column types
    .csv(file_path)
)

df_raw

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [15]:
df_raw.show(10, truncate=False)

+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                        |Quantity|InvoiceDate   |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------------+--------+--------------+---------+----------+--------------+
|536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER |6       |12/1/2010 8:26|2.55     |17850     |United Kingdom|
|536365   |71053    |WHITE METAL LANTERN                |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84406B   |CREAM CUPID HEARTS COAT HANGER     |8       |12/1/2010 8:26|2.75     |17850     |United Kingdom|
|536365   |84029G   |KNITTED UNION FLAG HOT WATER BOTTLE|6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.     |6       |12/1/2010 8:26|3.39     |17850     |United Kingdom|
|536365   |22752    |SET 7 BABUSHKA NESTING BOXES       

In [16]:
df_raw.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [17]:
print("Row count in df_raw:", df_raw.count())

Row count in df_raw: 541909


26/01/28 14:21:06 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [20]:
from pyspark.sql import functions as F

df_nulls = df_raw.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c)
    for c in df_raw.columns
])

df_nulls.show(truncate=False)

ConnectionRefusedError: [Errno 61] Connection refused

In [19]:
cols_of_interest = ["InvoiceNo", "StockCode", "Description", "Quantity", "UnitPrice", "InvoiceDate", "Country"]

df = df_raw.select(*cols_of_interest)

df.show(10, truncate=False)
df.printSchema()

ConnectionRefusedError: [Errno 61] Connection refused